<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [7]:
import pandas as pd 
import tweepy

In [8]:
keys = pd.read_csv("keys.csv", header=None)

In [9]:
# Leer las llaves
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [15]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [16]:
#TEMA: COVID 19

tweets = tweepy.Cursor(api.search,
                       q=['covid 19 -RT'],
                       lang='es',
                       geocode=f'4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [17]:
# Objeto generador
t = [tweet for tweet in tweets]
#t[0]

In [18]:
t_processed = []

for tweet in t:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t_processed.append(temp)

In [20]:
data = pd.DataFrame(t_processed)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data

,tweet,date,city,user_location,id,link
0,"De acuerdo con @JFernandeznino, otro metanális...",2021-05-14 01:08:23,Bogotá,"Bogotá, D.C., Colombia",1393010266829426689,https://twitter.com/i/web/status/1393010266829...
1,"Si en Colombia, personas como usted .@AlvaroUr...",2021-05-14 01:06:00,Bogotá,"Bogota, Colombia",1393009667157250048,https://twitter.com/i/web/status/1393009667157...
2,@sergiobarbosach @claudiablumc Mientras los co...,2021-05-14 01:03:23,Bogotá,"Bogotá, D.C., Colombia",1393009010975158272,https://twitter.com/i/web/status/1393009010975...
3,@CarlosFGalan Mientras los congresistas sesion...,2021-05-14 01:01:48,Bogotá,"Bogotá, D.C., Colombia",1393008609945075718,https://twitter.com/i/web/status/1393008609945...
4,@tavo2366 Mientras los congresistas sesionan v...,2021-05-14 00:59:18,Bogotá,"Bogotá, D.C., Colombia",1393007982791860230,https://twitter.com/i/web/status/1393007982791...
...,...,...,...,...,...,...
95,Nuevamente se disparan los contagios por covid...,2021-05-13 22:32:19,Bogotá,Ibagué Tolima,1392970991400325126,https://twitter.com/i/web/status/1392970991400...
96,Reporte Covid 19 para hoy 13 de Mayo 2021 htt...,2021-05-13 22:31:38,Bogotá,Acacías Meta,1392970821900111875,https://twitter.com/i/web/status/1392970821900...
97,De acuerdo al más reciente reporte del Ministe...,2021-05-13 22:31:19,Bogotá,Boyacá - Colombia,1392970741642055689,https://twitter.com/i/web/status/1392970741642...
98,@Brigittelgb @dario_maldonado Están ambiguo lo...,2021-05-13 22:30:51,Bogotá,"Bogotá, D.C., Colombia",1392970624176476163,https://twitter.com/i/web/status/1392970624176...


In [42]:
import snscrape.modules.twitter as sntwitter

import snscrape.modules.twitter as sntwitter

In [43]:
maxTweets = 100
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#Covid 19 ' + 'since:2014-07-15 until:2014-08-15').get_items()) :
        if i > maxTweets :
            break
        print(tweet)
        print(tweet.date)
        print(tweet.content)
        print("*"*100)
        print()

In [44]:
data = data[['tweet', 'date']]
data.head()

,tweet,date
0,"De acuerdo con @JFernandeznino, otro metanális...",2021-05-14 01:08:23
1,"Si en Colombia, personas como usted .@AlvaroUr...",2021-05-14 01:06:00
2,@sergiobarbosach @claudiablumc Mientras los co...,2021-05-14 01:03:23
3,@CarlosFGalan Mientras los congresistas sesion...,2021-05-14 01:01:48
4,@tavo2366 Mientras los congresistas sesionan v...,2021-05-14 00:59:18


In [45]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [46]:
data[['texto_limpio','links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweet,date,texto_limpio,links,hashtags,mentions
0,"De acuerdo con @JFernandeznino, otro metanális...",2021-05-14 01:08:23,De acuerdo con otro metanálisis indicó que lo...,[https://t.co/kivbuPrrHB],[],"[@JFernandeznino,]"
1,"Si en Colombia, personas como usted .@AlvaroUr...",2021-05-14 01:06:00,"Si en Colombia, personas como usted .@AlvaroUr...",[https://t.co/IZWZdrg0hA],[#Covid_19],[]
2,@sergiobarbosach @claudiablumc Mientras los co...,2021-05-14 01:03:23,Mientras los congresistas sesionan virtualme...,[],[],"[@sergiobarbosach, @claudiablumc]"
3,@CarlosFGalan Mientras los congresistas sesion...,2021-05-14 01:01:48,Mientras los congresistas sesionan virtualmen...,[],[],[@CarlosFGalan]
4,@tavo2366 Mientras los congresistas sesionan v...,2021-05-14 00:59:18,Mientras los congresistas sesionan virtualmen...,[],[],[@tavo2366]
...,...,...,...,...,...,...
95,Nuevamente se disparan los contagios por covid...,2021-05-13 22:32:19,Nuevamente se disparan los contagios por covid...,"[https://t.co/52vYJnSG0o, https://t.co/ra4J2pr...",[],[]
96,Reporte Covid 19 para hoy 13 de Mayo 2021 htt...,2021-05-13 22:31:38,Reporte Covid 19 para hoy 13 de Mayo 2021 a ...,[https://t.co/gCebDojlyU],[],[@elconucoacacias]
97,De acuerdo al más reciente reporte del Ministe...,2021-05-13 22:31:19,De acuerdo al más reciente reporte del Ministe...,[],[],[]
98,@Brigittelgb @dario_maldonado Están ambiguo lo...,2021-05-13 22:30:51,"Están ambiguo lo que dice, porque mi hermana...",[],[],"[@Brigittelgb, @dario_maldonado]"


In [47]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  

In [48]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweet,date,texto_limpio,links,hashtags,mentions,emojis
0,"De acuerdo con @JFernandeznino, otro metanális...",2021-05-14 01:08:23,De acuerdo con otro metanálisis indicó que lo...,[https://t.co/kivbuPrrHB],[],"[@JFernandeznino,]",[]
1,"Si en Colombia, personas como usted .@AlvaroUr...",2021-05-14 01:06:00,"Si en Colombia, personas como usted .@AlvaroUr...",[https://t.co/IZWZdrg0hA],[#Covid_19],[],[]
2,@sergiobarbosach @claudiablumc Mientras los co...,2021-05-14 01:03:23,Mientras los congresistas sesionan virtualme...,[],[],"[@sergiobarbosach, @claudiablumc]",[]
3,@CarlosFGalan Mientras los congresistas sesion...,2021-05-14 01:01:48,Mientras los congresistas sesionan virtualmen...,[],[],[@CarlosFGalan],[]
4,@tavo2366 Mientras los congresistas sesionan v...,2021-05-14 00:59:18,Mientras los congresistas sesionan virtualmen...,[],[],[@tavo2366],[]


In [49]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
47,00:01:00,1
60,00:05:00,1
11,00:06:00,2
63,00:08:00,1
26,00:09:00,1
...,...,...
21,23:54:00,1
65,23:55:00,1
51,23:56:00,1
25,23:58:00,1


In [50]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [51]:
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [52]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})
layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)